In [ ]:
!pip install transformers
!pip install pypinyin
!pip install jieba
!pip install paddlepaddle

In [1]:
import re,time,json
from collections import defaultdict
from torch.utils.data import DataLoader
from pypinyin import pinyin, Style
from tqdm import tqdm
import pickle

import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from copy import deepcopy

from transformers import (BertTokenizer,BertConfig,BertModel)

from model.Embedding import FusionEmbedding,GlyphEmbedding,PinyinEmbedding
from model.fusionDataset import FusionDataset
import torch.optim as optim

import jieba
import jieba.posseg as pseg
import paddle

config = BertConfig.from_pretrained('AnchiBERT')
tokenizer = BertTokenizer.from_pretrained('AnchiBERT')
Anchibert = BertModel.from_pretrained('AnchiBERT',config=config)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

c:\users\user\anaconda3\envs\py36\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\user\anaconda3\envs\py36\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
c:\users\user\anaconda3\envs\py36\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)
Some weights of the model checkpoint at AnchiBERT were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are ini

### Example for glyph + pinyin embedding

In [2]:
glyph = GlyphEmbedding('data/glyph_weight.npy')
pinyinV = PinyinEmbedding(30,'data/pinyin_map.json')
# Example
display(glyph(torch.tensor([0,1,2])))
display(pinyinV(torch.tensor([0,1])))

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0720, -0.0596,  0.0230,  ...,  0.0150, -0.0975, -0.0763],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       grad_fn=<EmbeddingBackward>)

tensor([[ 0.2357, -0.2738, -0.2652,  0.1372, -0.0071, -0.3078,  0.0094,  0.0898,
         -0.1926, -0.2237,  0.1600,  0.1161,  0.2236, -0.0088,  0.1961, -0.2132,
          0.2560,  0.2003, -0.0597,  0.0148,  0.1607,  0.1465,  0.0915, -0.0037,
         -0.2926,  0.2344,  0.2502,  0.0166, -0.0071,  0.0503],
        [-0.1495, -0.2923, -0.2148, -0.2737, -0.1621,  0.2301,  0.2625,  0.1725,
         -0.2636, -0.0225,  0.2649,  0.3154, -0.1482,  0.1330,  0.1423,  0.1166,
          0.2852,  0.1410,  0.1775, -0.1640,  0.0467, -0.0750,  0.2550, -0.0938,
          0.1038,  0.1584,  0.1917,  0.0675, -0.2958, -0.1075]],
       grad_fn=<EmbeddingBackward>)

In [3]:
# Example
_ = tokenizer('[UNK]辛业怠π[PAD]',return_tensors='pt')
display(_)
Anchibert(**_).last_hidden_state.shape

{'input_ids': tensor([[ 101,  100, 6789,  689, 2591,  223,    0,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

torch.Size([1, 8, 768])

In [4]:
tokenizer.convert_ids_to_tokens(_['input_ids'][0,:])

['[CLS]', '[UNK]', '辛', '业', '怠', 'π', '[PAD]', '[SEP]']

### Load Necessary preproceeded Data

In [5]:
with open('data/char_map.json','r') as f:
    glyph2ix = defaultdict(lambda : 1)
    glyph2ix.update({'[CLS]':0,'[SEP]':0,'[PAD]':0})
    glyph2ix.update(json.load(f))

with open('data/pinyin_map.json','r') as f:
    pinyin2ix = defaultdict(lambda : 1)
    pinyin2ix.update({'[CLS]':0,'[SEP]':0,'[PAD]':0})
    pinyin2ix.update(json.load(f))
    
with open('data/pos_tags.json','r') as f:
    pos2ix = defaultdict(lambda : 0)
    pos2ix.update(json.load(f))

In [6]:
# train 上联
with open("couplet/train/in.txt",encoding='utf8') as f:
    tr_in =  [row.strip().split() for row in f.readlines()]
# train 下联  
with open("couplet/train/out.txt",encoding='utf8') as f:
    tr_out = [row.strip().split() for row in f.readlines()]
with open('data/train_in_pos.pt','rb') as f:
    tr_pos_in = pickle.load(f)
with open('data/train_out_pos.pt','rb') as f:
    tr_pos_out = pickle.load(f)

In [31]:
split_idx = int(len(tr_in) * 0.9)
trainSet = FusionDataset(tr_in[:split_idx],tokenizer,glyph2ix,pinyin2ix,pos2ix,tr_out[:split_idx],tr_pos_in[:split_idx],tr_pos_out[:split_idx])
valSet = FusionDataset(tr_in[split_idx:],tokenizer,glyph2ix,pinyin2ix,pos2ix,tr_out[split_idx:],tr_pos_in[split_idx:],tr_pos_out[split_idx:])

693441it [04:58, 2326.35it/s]
693441it [05:03, 2284.49it/s]
77050it [00:34, 2202.34it/s]
77050it [00:33, 2307.76it/s]


In [35]:
batch_size = 64
trainLoader = DataLoader(trainSet,batch_size=batch_size,shuffle=True)
validLoader = DataLoader(valSet,batch_size=batch_size,shuffle=False)

In [12]:
from model.fusion_transformer import Fusion_Anchi_Trans_Decoder, Fusion_Anchi_Transformer

In [13]:
config = { # for Fusion_Anchi_Trans_Decoder
    'max_position_embeddings':50,
    'hidden_size':768,
    'font_weight_path':'data/glyph_weight.npy',
    'pinyin_embed_dim':30,
    'pinyin_path':'data/pinyin_map.json',
    'tag_size':30,
    'tag_emb_dim':10,
    'layer_norm_eps':1e-12,
    'hidden_dropout':0.1,
    'nhead':12,
    'num_layers':6,
    'output_dim':21128 # fixed
}
config2 = { # Fusion_Anchi_Transformer
    'max_position_embeddings':50,
    'hidden_size':768,
    'font_weight_path':'data/glyph_weight.npy',
    'pinyin_embed_dim':30,
    'pinyin_path':'data/pinyin_map.json',
    'tag_size':30,
    'tag_emb_dim':10,
    'layer_norm_eps':1e-12,
    'hidden_dropout':0.1,
    'nhead':12,
    'num_encoder_layers':6,
    'num_decoder_layers':6,
    'output_dim':21128, # fixed
    'dim_feedforward': 3072,
    'activation':'relu',
    'trans_dropout':0.1
}

In [14]:
model = Fusion_Anchi_Trans_Decoder(config).to(device)

In [15]:
model2 = Fusion_Anchi_Transformer(config2).to(device)

In [ ]:
# model3 = FusionEmbedding(config)
# model3.to(device)

FusionEmbedding(
  (position_embeddings): Embedding(50, 768)
  (glyph_embeddings): GlyphEmbedding(
    (embedding): Embedding(9110, 576)
  )
  (pinyin_embeddings): PinyinEmbedding(
    (embedding): Embedding(1297, 30, padding_idx=0)
  )
  (pos_tag_embeddings): Embedding(30, 10, padding_idx=0)
  (fc): Linear(in_features=1384, out_features=768, bias=True)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [62]:
epochs = 10
lr = 0.01
save_model_name = 'decoder_md.pt'

model = Fusion_Anchi_Trans_Decoder(config).to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(),lr=lr)


train_net(model, Anchibert, [trainLoader, validLoader], epochs, save_model_name, optimizer, criterion, device = 'cpu')

KeyboardInterrupt: 

In [59]:
def epoch_train(net, Anchibert, dataLoader, optimizer, criterion, device = 'cuda0', mode = 'train'):
    valid_loss = 0.0
    for data in dataLoader:
        data = [data_sub.to(device) for data_sub in data]
        
        Xsents_input_ids,Xsents_token_type_ids,\
        Xsents_attention_mask,Xsents_pinyin_ids,\
        Xsents_glyph_ids,Xsents_pos_ids,\
        Ysents_input_ids,Ysents_token_type_ids,\
        Ysents_attention_mask,Ysents_pinyin_ids,\
        Ysents_glyph_ids,Ysents_pos_ids = data

        
        Xword_embeddings = Anchibert(Xsents_input_ids,      \
                                     Xsents_token_type_ids, \
                                     Xsents_attention_mask  \
                                    )['last_hidden_state'].detach()


        Yword_embeddings = Anchibert(Ysents_input_ids,      \
                                     Ysents_token_type_ids, \
                                     Ysents_attention_mask  \
                                    )['last_hidden_state'].detach()
        
        # outputs1 for Fusion_Anchi_Trans_Decoder
        outputs1 = model(Xword_embeddings,Xsents_pinyin_ids, \
                        Xsents_token_type_ids,Xsents_pos_ids,\
                        Yword_embeddings,Ysents_pinyin_ids, \
                        Ysents_token_type_ids,Ysents_pos_ids,\
                        Xsents_attention_mask.bool(),Ysents_attention_mask.bool())

        # get the true model output label from decoder input id
        trueY = torch.zeros(Ysents_input_ids.shape,dtype=torch.long,device=device)
        trueY[:,:-1] = Ysents_input_ids[:,1:]
        # find the index of first padding return 0 if no_padding 
        indices = torch.argmin(Ysents_attention_mask,dim=1).tolist()

        for i, index in enumerate(indices):
            trueY[i,index-1] = 102
        trueY = trueY.view(-1)

        outputs1 = outputs1.view(-1,outputs1.shape[-1])    
        loss = criterion(outputs1,trueY)
        
        if mode == 'train':
            # backward
            loss.backward()
            optimizer.step()
        else:
            valid_loss += loss.item()*data[0].size(0)
            
    valid_loss = valid_loss/len(dataLoader.dataset)
    return valid_loss



In [60]:
def train_net(net, Anchibert, data, num_epochs, save_model_name, optimizer, criterion, device = 'cuda0', valid_loss_min = np.Inf):
    train_data = data[0]
    valid_data = data[1]
    
    net = net.to(device)
    Anchibert = Anchibert.to(device)
        
    for i in range(num_epochs):
        net = net.train()
        _ = epoch_train(net, Anchibert, train_data, optimizer, criterion, device)

        net = net.eval()
        valid_loss = epoch_train(net, Anchibert, train_data, optimizer, criterion, device, mode = 'valid')
        
        
        print(f'epoch = {i}, loss = {valid_loss}')
        if valid_loss <= valid_loss_min:
            print(f'epoch = {i}')
            print('Validation loss ({:.4f} --> {:.4f}).'.format(
            valid_loss_min,
            valid_loss))

            torch.save(net.state_dict(), save_model_name)
            valid_loss_min = valid_loss


In [24]:
Anchibert.to(device)
for Xsents_input_ids,Xsents_token_type_ids,\
    Xsents_attention_mask,Xsents_pinyin_ids,\
    Xsents_glyph_ids,Xsents_pos_ids,\
    Ysents_input_ids,Ysents_token_type_ids,\
    Ysents_attention_mask,Ysents_pinyin_ids,\
    Ysents_glyph_ids,Ysents_pos_ids in trainLoader:
    
    Xsents_input_ids = Xsents_input_ids.to(device)
    Xsents_token_type_ids = Xsents_token_type_ids.to(device)
    Xsents_attention_mask = Xsents_attention_mask.to(device)
    Xsents_pinyin_ids = Xsents_pinyin_ids.to(device)
    Xsents_glyph_ids = Xsents_glyph_ids.to(device)
    Xsents_pos_ids = Xsents_pos_ids.to(device)
    Ysents_input_ids = Ysents_input_ids.to(device)
    Ysents_token_type_ids = Ysents_token_type_ids.to(device)
    Ysents_attention_mask = Ysents_attention_mask.to(device)
    Ysents_pinyin_ids = Ysents_pinyin_ids.to(device)
    Ysents_glyph_ids = Ysents_glyph_ids.to(device)
    Ysents_pos_ids = Ysents_pos_ids.to(device)
    
    Xword_embeddings = Anchibert(Xsents_input_ids,      \
                                 Xsents_token_type_ids, \
                                 Xsents_attention_mask  \
                                )['last_hidden_state'].detach()


    Yword_embeddings = Anchibert(Ysents_input_ids,      \
                                 Ysents_token_type_ids, \
                                 Ysents_attention_mask  \
                                )['last_hidden_state'].detach()
    # outputs1 for Fusion_Anchi_Trans_Decoder
    outputs1 = model(Xword_embeddings,Xsents_pinyin_ids, \
                    Xsents_token_type_ids,Xsents_pos_ids,\
                    Yword_embeddings,Ysents_pinyin_ids, \
                    Ysents_token_type_ids,Ysents_pos_ids,\
                    Xsents_attention_mask.bool(),Ysents_attention_mask.bool())
    
    # outputs2 Fusion_Anchi_Transformer(config2)
#     outputs2= model2(Xword_embeddings,Xsents_pinyin_ids, \
#                     Xsents_token_type_ids,Xsents_pos_ids,\
#                     Yword_embeddings,Ysents_pinyin_ids, \
#                     Ysents_token_type_ids,Ysents_pos_ids,\
#                     Xsents_attention_mask.bool(),Ysents_attention_mask.bool())
    
    # FusionEmbedding
#     outputembeddingsX = model3(Xword_embeddings,Xsents_pinyin_ids, \
#                          Xsents_token_type_ids,Xsents_pos_ids)
#     outputembeddingsY = model3(Yword_embeddings,Ysents_pinyin_ids, \
#                              Ysents_token_type_ids,Ysents_pos_ids)
    
    
    # get the true model output label from decoder input id
    trueY = torch.zeros(Ysents_input_ids.shape,dtype=torch.long,device=device)
    trueY[:,:-1] = Ysents_input_ids[:,1:]
    # find the index of first padding return 0 if no_padding 
    indices = torch.argmin(Ysents_attention_mask,dim=1).tolist()

    for i, index in enumerate(indices):
        trueY[i,index-1] = 102
    trueY = trueY.view(-1)
    
    outputs1 = outputs1.view(-1,outputs1.shape[-1])    
    loss = criterion(outputs1,trueY)
    loss.backward()
    break

In [41]:
trueY.size()

torch.Size([1792])

In [ ]:
outputs.shape, outputs2.shape, outputembeddingsX.shape,outputembeddingsY.shape

### Way to generate new decoder input from raw sentence 

记得把model output 中 不存在 glyph dict的词以及'[SEP]' 和'[PAD]'给转换成 '_'。

In [21]:
sents = tr_out[:2]
sents

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['晨', '露', '润', '花', '花', '更', '红'], ['万', '方', '乐', '奏', '有', '于', '阗']]

In [24]:
FusionDataset.prepare_sequence(sents=sents, tokenizer=tokenizer,
                               glyph2ix=glyph2ix,pinyin2ix=pinyin2ix,
                               pos2ix=pos2ix,encode=False,skip_error=False)

Paddle enabled successfully......
DEBUG:jieba._compat:Paddle enabled successfully......
2it [00:00, 83.52it/s]


([tensor([ 101, 3247, 7463, 3883, 5709, 5709, 3291, 5273], dtype=torch.int32),
  tensor([ 101,  674, 3175,  727, 1941, 3300,  754,  100], dtype=torch.int32)],
 [tensor([0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32),
  tensor([0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)],
 [tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.int32),
  tensor([1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.int32)],
 [tensor([  0, 717,  59, 873, 579, 579, 217, 334], dtype=torch.int32),
  tensor([   0,  187,  476,  501,  807, 1178,  263, 1002], dtype=torch.int32)],
 [tensor([   0, 5656, 1568, 3172, 3915, 3915, 3367, 8111], dtype=torch.int32),
  tensor([   0, 1729, 3171, 4526, 2058, 8413, 3025, 6866], dtype=torch.int32)],
 [tensor([ 0,  4,  4,  4,  4,  4, 16, 12], dtype=torch.int32),
  tensor([ 0, 26, 26, 26, 26,  9, 27, 27], dtype=torch.int32)])